In [1]:
import re
import json
import time
import os
import sys
import numpy as np
import qiskit

from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.converters import circuit_to_dag

# Dynamically add the project root to sys.path
project_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd())))  # Adjust path to root
sys.path.append(project_root)

In [2]:
from src.CE_module import construct_qcc_circuit, CE_recur_tree
from src.CA_module import extract_CNOT_network, update_probabilities
from src.utilities import compare_lists, generate_pauli_strings
from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers

In [3]:
#QAOA for MAXCUT 
cost_hamiltonian = ['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII']
mixer_hamiltonian = ['XIIII', 'IXIII', 'IIXII', 'IIIXI', 'IIIIX']
test_paulis = (cost_hamiltonian + mixer_hamiltonian)* 2
test_params = [3.271] * 6 +   [2.874] * 5 + [2.641] * 6 + [0.9339]* 5

start_time = time.time()
opt_qc, append_clifford, sorted_entanglers = CE_recur_tree(entanglers=test_paulis, params=test_params, barrier=False)
opt_qiskit = transpile(opt_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])

#need to add a layer of hadamard gates:
for i in range(0, len(test_paulis[0])):
    opt_qc.h(i)
opt_qc.measure_active()
end_time = time.time()
combined_time = end_time - start_time


origin_qc = construct_qcc_circuit(entanglers = test_paulis, params = test_params, barrier=False)
#generate the qiskit optimized circuit
origin_qiskit = transpile(origin_qc, optimization_level = 3, basis_gates = ["cx", "sx", "x", "rz"])
origin_qiskit.measure_active() 
origin_qiskit.count_ops()['cx']

24

In [4]:
append_clifford.draw()

┌───┐                     
q_0: ────────────■──┤ H ├─X───■────■────■─────
               ┌─┴─┐├───┤ │   │  ┌─┴─┐  │     
q_1: ───────■──┤ X ├┤ H ├─┼───┼──┤ X ├──┼───X─
          ┌─┴─┐├───┤└───┘ │   │  └───┘┌─┴─┐ │ 
q_2: ──■──┤ X ├┤ H ├──────┼───┼───────┤ X ├─X─
       │  ├───┤└───┘      │   │       └───┘   
q_3: ──┼──┤ H ├───────────┼───┼───────────────
     ┌─┴─┐├───┤           │ ┌─┴─┐             
q_4: ┤ X ├┤ H ├───────────X─┤ X ├─────────────
     └───┘└───┘             └───┘

In [5]:
opt_qc.count_ops()

OrderedDict([('cx', 22),
             ('rz', 22),
             ('h', 20),
             ('measure', 5),
             ('barrier', 1)])

In [6]:
opt_qc.draw()

»
      q_0: ──■─────────────────■─────────────────■─────────────────■──»
           ┌─┴─┐┌───────────┐  │                 │      ┌───┐      │  »
      q_1: ┤ X ├┤ Rz(3.271) ├──┼─────────────────┼──────┤ X ├──────┼──»
           └───┘└───────────┘┌─┴─┐┌───────────┐  │      └─┬─┘      │  »
      q_2: ──────────────────┤ X ├┤ Rz(3.271) ├──┼────────■────────┼──»
                             └───┘└───────────┘  │               ┌─┴─┐»
      q_3: ──────────────────────────────────────┼───────────────┤ X ├»
                                               ┌─┴─┐┌───────────┐└───┘»
      q_4: ────────────────────────────────────┤ X ├┤ Rz(3.271) ├─────»
                                               └───┘└───────────┘     »
measure: 5/═══════════════════════════════════════════════════════════»
                                                                      »
«               ┌───┐                                       ┌───┐    »
«      q_0: ────┤ H ├───────────────────────────────────────┤ X ├────»
«           ┌───┴───┴───┐    ┌───┐    ┌───────────┐┌───┐    └─┬─┘    »
«      q_1: ┤ Rz(3.271) ├────┤ H ├────┤ Rz(2.874) ├┤ X ├──────┼──────»
«           └───┬───┬───┘┌───┴───┴───┐└───┬───┬───┘└─┬─┘      │      »
«      q_2: ────┤ X ├────┤ Rz(3.271) ├────┤ H ├──────■────────┼──────»
«               └─┬─┘    └───────────┘    ├───┤               │      »
«      q_3: ──────■────────────■──────────┤ H ├───────────────■──────»
«                            ┌─┴─┐    ┌───┴───┴───┐┌───┐┌───────────┐»
«      q_4: ─────────────────┤ X ├────┤ Rz(3.271) ├┤ H ├┤ Rz(2.874) ├»
«                            └───┘    └───────────┘└───┘└───────────┘»
«measure: 5/═════════════════════════════════════════════════════════»
«                                                                    »
«           ┌───────────┐┌───┐                               ┌───┐┌───────────┐»
«      q_0: ┤ Rz(2.874) ├┤ H ├───────────────■───────────────┤ X ├┤ Rz(2.641) ├»
«           ├───────────┤├───┤┌───────────┐  │               └─┬─┘└───┬───┬───┘»
«      q_1: ┤ Rz(2.874) ├┤ H ├┤ Rz(2.641) ├──┼─────────────────┼──────┤ X ├────»
«           └───────────┘├───┤└───────────┘┌─┴─┐┌───────────┐  │      └─┬─┘    »
«      q_2: ──────■──────┤ H ├─────────────┤ X ├┤ Rz(2.641) ├──┼────────■──────»
«               ┌─┴─┐    ├───┤┌───────────┐├───┤└───────────┘  │               »
«      q_3: ────┤ X ├────┤ X ├┤ Rz(2.874) ├┤ H ├───────────────┼────────■──────»
«               └───┘    └─┬─┘└───┬───┬───┘└───┘               │      ┌─┴─┐    »
«      q_4: ───────────────■──────┤ H ├────────────────────────■──────┤ X ├────»
«                                 └───┘                               └───┘    »
«measure: 5/═══════════════════════════════════════════════════════════════════»
«                                                                              »
«                                 ┌───┐     ┌────────────┐                  »
«      q_0: ───────────────■──────┤ H ├─────┤ Rz(0.9339) ├──────────────────»
«           ┌───────────┐  │      └───┘     └────────────┘    ┌───┐         »
«      q_1: ┤ Rz(2.641) ├──┼──────────────────────■───────────┤ H ├─────────»
«           └───┬───┬───┘  │  ┌────────────┐      │           └───┘         »
«      q_2: ────┤ H ├──────┼──┤ Rz(0.9339) ├──────┼─────────────────────────»
«               ├───┤      │  └────────────┘      │                         »
«      q_3: ────┤ H ├──────┼──────────────────────┼─────────────────────────»
«           ┌───┴───┴───┐┌─┴─┐                  ┌─┴─┐     ┌───────────┐┌───┐»
«      q_4: ┤ Rz(2.641) ├┤ X ├──────────────────┤ X ├─────┤ Rz(2.641) ├┤ H ├»
«           └───────────┘└───┘                  └───┘     └───────────┘└───┘»
«measure: 5/════════════════════════════════════════════════════════════════»
«                                                                           »
«                                                      ┌───┐                   »
«      q_0: ────────────────────────────────────────■──┤ H ├───

In [7]:
append_clifford.draw()

┌───┐                     
q_0: ────────────■──┤ H ├─X───■────■────■─────
               ┌─┴─┐├───┤ │   │  ┌─┴─┐  │     
q_1: ───────■──┤ X ├┤ H ├─┼───┼──┤ X ├──┼───X─
          ┌─┴─┐├───┤└───┘ │   │  └───┘┌─┴─┐ │ 
q_2: ──■──┤ X ├┤ H ├──────┼───┼───────┤ X ├─X─
       │  ├───┤└───┘      │   │       └───┘   
q_3: ──┼──┤ H ├───────────┼───┼───────────────
     ┌─┴─┐├───┤           │ ┌─┴─┐             
q_4: ┤ X ├┤ H ├───────────X─┤ X ├─────────────
     └───┘└───┘             └───┘

In [9]:
cnot_network = extract_CNOT_network(append_clifford)

In [10]:
#Run the original circuit on a noiseless simulator:
simulator = AerSimulator()
origin_result = simulator.run(origin_qiskit, shots = 10000).result()

In [11]:
def get_top_k_values(input_dict, k):
    # Sort the dictionary by values in descending order and return the top k items
    sorted_items = sorted(input_dict.items(), key=lambda item: item[1], reverse=True)
    return sorted_items[:k]

In [12]:
origin_probs= origin_result.get_counts()

In [13]:
top_k_states_origin = get_top_k_values(origin_probs, 20)
print(top_k_states_origin)

[('11111', 3556), ('11110', 1045), ('11011', 1029), ('11101', 697), ('10111', 658), ('01111', 652), ('11010', 344), ('10110', 206), ('01011', 205), ('11001', 202), ('11100', 191), ('01110', 174), ('10011', 159), ('01101', 151), ('10101', 147), ('00111', 137), ('10010', 64), ('01010', 59), ('00011', 43), ('00101', 42)]


In [14]:
#Run the optimized circuit on a noiseless simulator:

opt_result = simulator.run(opt_qc, shots = 10000).result()

In [15]:
dag = circuit_to_dag(cnot_network)
updated_states = {}

updated_probs = update_probabilities(opt_result.get_counts(), dag)


In [16]:
from qiskit.quantum_info.analysis import hellinger_fidelity

In [17]:
hellinger_fidelity(origin_probs, updated_probs)

0.9975860784733286

In [18]:
top_k_states_origin = get_top_k_values(origin_probs, 20)
print(top_k_states_origin)

[('11111', 3556), ('11110', 1045), ('11011', 1029), ('11101', 697), ('10111', 658), ('01111', 652), ('11010', 344), ('10110', 206), ('01011', 205), ('11001', 202), ('11100', 191), ('01110', 174), ('10011', 159), ('01101', 151), ('10101', 147), ('00111', 137), ('10010', 64), ('01010', 59), ('00011', 43), ('00101', 42)]


In [19]:
top_k_states_updated = get_top_k_values(updated_probs, 20)
print(top_k_states_updated)

[('11111', 3528), ('11110', 1067), ('11011', 1053), ('11101', 720), ('01111', 670), ('10111', 626), ('11010', 434), ('01011', 213), ('10110', 195), ('00111', 188), ('11001', 177), ('11100', 171), ('01110', 164), ('10011', 144), ('01101', 125), ('10101', 115), ('11000', 52), ('01010', 48), ('00110', 45), ('10010', 43)]
